In [1]:
import geopandas, pandas


da_2021 = "../Data/lda_000b21a_e/lda_000b21a_e.shp"
da_2006 = "../Data/gda_000b06a_e/gda_000b06a_e.shp"
csd_2021 = "../Data/lcsd000b21a_e/lcsd000b21a_e.shp"

canada_da_21 = geopandas.read_file(da_2021)
canada_da_06 = geopandas.read_file(da_2006)
canada_csd_21 = geopandas.read_file(csd_2021)

ontario_da_21 = canada_da_21[canada_da_21['PRUID'] == '35']
ontario_da_06 = canada_da_06[canada_da_06['PRUID'] == '35']
ontario_csd_21 = canada_csd_21[canada_csd_21['PRUID'] == '35']

region_summary = "../../../ggh-growth/regions-summary.csv"
region_summary = pandas.read_csv(region_summary)



KeyboardInterrupt: 

Bridging the Gap Between CSDUID and DAUID


In [3]:
#import census reference table, which contains the DGUID for DA and CSD level. 
DA_to_CSD = "../Data/2021_DA_CSDUID_Reference_Table.csv"
DA_to_CSD = pandas.read_csv(DA_to_CSD)
#merge the reference table to the Ontario DA file for 2021
DA_with_CSDDGUID = pandas.merge(ontario_da_21, DA_to_CSD, left_on = "DGUID", right_on = "DADGUID_ADIDUGD")
#copy the 'DGUID', 'CSDUID' to a new dataframe called DA_with_CSD, then merge it to DA_with_CSDDGUID
DA_with_CSD = ontario_csd_21[['DGUID', 'CSDUID']].copy()
GGH_DA_with_CSDUID_21 = pandas.merge(DA_with_CSDDGUID, DA_with_CSD,  left_on= "CSDDGUID_SDRIDUGD", right_on= "DGUID")
#remove redundant fields 
GGH_DA_with_CSDUID_21 = GGH_DA_with_CSDUID_21.drop(["CSDDGUID_SDRIDUGD", "DADGUID_ADIDUGD", "DGUID_y", "DGUID_x"], axis = 1)
GGH_DA_with_CSDUID_06 = ontario_da_06.drop(["CDUID", "ERUID", "CCSUID", "CTUID", "CMAUID", "PRUID"], axis = 1)
#canada_da_21.drop('CensusID2', axis =1)
#turn the region summary into a list
censusIDList = region_summary['CSDUID2'].tolist()
municipalityList = region_summary['Municipality'].tolist()
macroList = region_summary["Macro"].tolist()
#loop through the list to add a new field called CSDID2, CSDID2 will be a new key for joining with regions-summary.csv
i = 0
while i < len(censusIDList):
    regionID = censusIDList[i]
    macro = macroList[i]
    municipality = municipalityList[i]
    GGH_DA_with_CSDUID_21.loc[GGH_DA_with_CSDUID_21['CSDUID'].str.startswith(f'{regionID}'), "Macro"] = macro
    GGH_DA_with_CSDUID_06.loc[GGH_DA_with_CSDUID_06['CSDUID'].str.startswith(f'{regionID}'), "Macro"] = macro
    GGH_DA_with_CSDUID_21.loc[GGH_DA_with_CSDUID_21['CSDUID'].str.startswith(f'{regionID}'), "Municipality"] = municipality
    GGH_DA_with_CSDUID_06.loc[GGH_DA_with_CSDUID_06['CSDUID'].str.startswith(f'{regionID}'), "Municipality"] = municipality
    i+=1

GGH_DA_with_CSDUID_21 = GGH_DA_with_CSDUID_21.dropna()
GGH_DA_with_CSDUID_06 = GGH_DA_with_CSDUID_06.dropna()


Joining Census Data With the Census Boundary Files

In [4]:
#export to shapefile
GGH_DA_with_CSDUID_21.to_file("../Export/GGH_DA_with_CSDUID_2021.geojson")
GGH_DA_with_CSDUID_06.to_file("../Export/GGH_DA_with_CSDUID_2006.geojson")

In [ ]:

census_ontario_data_06 = "../Data/Census Data/Census2006.csv"
census_ontario_data_06 = pandas.read_csv(census_ontario_data_06, encoding='utf-8')
census_ontario_data_21 = "../Data/Census Data/Census2021.csv"
census_ontario_data_21 = pandas.read_csv(census_ontario_data_21, encoding='utf-8')

In [ ]:

GGH_DA_with_CSDUID_06 = "../Export/GGH_DA_with_CSDUID_2006.geojson"
GGH_DA_with_CSDUID_06 = geopandas.read_file(GGH_DA_with_CSDUID_06)
GGH_DA_with_CSDUID_21 = "../Export/GGH_DA_with_CSDUID_2021.geojson"
GGH_DA_with_CSDUID_21 = geopandas.read_file(GGH_DA_with_CSDUID_21)

GGH_DA_with_CSDUID_06["DAUID"] = GGH_DA_with_CSDUID_06["DAUID"].astype('str')
census_ontario_data_06["Dauid"] = census_ontario_data_06["Dauid"].astype('str')
GGH_Census_06 = pandas.merge(GGH_DA_with_CSDUID_06, census_ontario_data_06,  left_on= "DAUID", right_on= "Dauid")

GGH_DA_with_CSDUID_21["DAUID"] = GGH_DA_with_CSDUID_21["DAUID"].astype('str')
census_ontario_data_21["GEO UID"] = census_ontario_data_21["GEO UID"].astype('str')
GGH_Census_21 = pandas.merge(GGH_DA_with_CSDUID_21, census_ontario_data_21,  left_on= "DAUID", right_on= "GEO UID")
GGH_Census_06 = GGH_Census_06.drop('Dauid', axis =1)
GGH_Census_21 = GGH_Census_21.drop(['LANDAREA', 'PRUID', 'GEO UID', 'Land area in square kilometres'], axis = 1)

GGH06_fields = GGH_Census_06.columns [5:]
GGH21_fields = GGH_Census_21.columns [5:]
GGH06_fields = GGH06_fields.tolist()
GGH21_fields = GGH21_fields.tolist()

GGH_Census_06.to_file("../Export/GGH_Census_06.geojson")
GGH_Census_21.to_file("../Export/GGH_Census_21.geojson")

In [ ]:
pivot_table_06 = pandas.pivot_table(data = GGH_Census_06, values = GGH06_fields, index = ['Municipality'], aggfunc = numpy.sum)
pivot_table_21 = pandas.pivot_table(data = GGH_Census_21, values = GGH21_fields, index = ['Municipality'], aggfunc = numpy.sum)
pivot_table_06 = pivot_table_06.reset_index()
pivot_table_21 = pivot_table_21.reset_index()
pivot_table = pivot_table_06['Municipality'].reset_index()

fieldNames = []
for f in GGH21_fields:
    fieldNames.append(f'Change - {f}')

i = 0
while i < len(GGH06_fields):
    pivot_table[fieldNames[i]] = pivot_table_21[GGH21_fields[i]] - pivot_table_06[GGH06_fields[i]]
    i+=1

pivot_table = pivot_table.drop(['index'], axis = 1)

ggh_regions = geopandas.read_file("../../../ggh-growth/ggh-growth-plan-regions.geojson")
ggh = pandas.merge(ggh_regions, pivot_table, right_on = "Municipality", left_on = "Name")
ggh.to_file("../Export/GGH_Census_Change.geojson")



In [ ]:
for field in GGH_Census_21.columns.tolist()[5:]:
    maps = GGH_Census_21.plot(column = field, legend = True, cmap='Purples', edgecolor="grey", linewidth = 0.01, scheme='quantiles', figsize = (10.0,8.0), aspect = "auto", legend_kwds={'bbox_to_anchor': (1.3, 1)})
    maps.set_axis_off()
    maps.set_title(field)

    output_file_name = f"../Export/Maps/2021_{field}.png"
    maps.get_figure().savefig(output_file_name, dpi = 600)

In [ ]:
for field in GGH_Census_06.columns.tolist()[5:]:
    maps = GGH_Census_06.plot(column = field, legend = True, cmap='Purples', edgecolor="grey", linewidth = 0.01, scheme='quantiles', figsize = (10.0,8.0), aspect = "auto", legend_kwds={'bbox_to_anchor': (1.3, 1)})
    maps.set_axis_off()
    maps.set_title(field)

    output_file_name = f"../Export/Maps/2006_{field}.png"
    maps.get_figure().savefig(output_file_name, dpi = 600)

In [ ]:
for field in ggh.columns.tolist()[3:]:
    maps = ggh.plot(column = field, legend = True, cmap='Purples', edgecolor="grey", linewidth = 0.4, scheme='quantiles', figsize = (10.0,8.0), aspect = "auto", legend_kwds={'bbox_to_anchor': (1.3, 1)})
    maps.set_axis_off()
    maps.set_title(field)

    output_file_name = f"../Export/Maps/Change_{field}.png"
    maps.get_figure().savefig(output_file_name, dpi = 600)
